In [3]:
import numpy as np
# help(np.linalg.norm)

In [25]:
def cal_Cosine_Similarity(x,y):
    print(x,y)
    # x = np.linalg.norm(x,ord=2,)
    return np.dot(x,y.T)/(np.linalg.norm(x,ord=2,)*np.linalg.norm(y,ord=2,))

x = np.arange(0,10,2)
y = np.arange(1,11,2)
out = cal_Cosine_Similarity(x,y)   
print(out)

[0 2 4 6 8] [1 3 5 7 9]
0.9949366763261819


# BN的计算过程：
## 训练阶段
![](./bn_train.jpg)

## 测试阶段
![](./bn_test.jpg)

**本案例说明：**

训练阶段：
1.假设输入的数据的格式是4行3列，4行数据分别输入到4个神经元。每个神经元处理3个batch的数据。
2.假定测试集和训练集具有同分布（均值、方差），在训练阶段需要计算训练集的均值和方差(采用滑动平均)。
3.在全局训练数据中看每个batch，不同的batch可能分布偏差很大，这样网络不好拟合数据不太稳定(covariate shift)，
需要捕捉到数据分布偏差的变化量，所以引入了伸缩因子γ和偏移因子β，这两个参数需要网络进行学习。

测试阶段：
输入的是一条条的数据

refer:
[Batch Normalization导读](https://blog.csdn.net/malefactor/article/details/51476961)

In [ ]:


def cal_bn( data,fn_grad,static_args,flag,eps):
    gamma = fn_grad["gamma"]
    beta = fn_grad["beta"]
    uB_momentum = static_args["uB_momentum"]
    varB_momentum = static_args["varB_momentum"]
    if flag == "train":
        uB = np.mean(x, axis=1,keepdims=True)
        varB = np.var(x, axis=1,keepdims=True)
        uB_momentum = (1-momentum)*uB_momentum + momentum*uB
        varB_momentum = (1-momentum)*varB_momentum + momentum*varB
        x_ = (x - uB)/(np.sqrt(varB+eps))
        y_ = gamma* x + beta
        return y_
    else:
        y_ = (gamma * x)/(np.sqrt(varB_momentum+eps))-(beta-gamma*uB_momentum/(np.sqrt(varB_momentum+eps)))
        return y_


x = np.arange(0,12,1)
x.shape = (4, 3) # 设batch为4
print(x)
eps =  0.0001   # 常量
# 在测试时，输入的是单个数据，又假定测试集和训练集具有同分布（均值、方差）
momentum = 0.1
uB_momentum = np.array([[1.1],[2.2],[3.3],[2.2]])
varB_momentum = np.array([[0.5],[0.75],[1.0],[1.5]])

# 伸缩因子用于减少训练和测试之间分布差异
gamma = np.array([[1.],[2.],[3.],[2.]])    # 需要学习的参数
beta = np.array([[1.5],[2.5],[3.5],[2.5]]) # 需要学习的参数

flag = "train" # 测试还是训练的标志位
fn_grad = {"gamma":gamma,"beta":beta}   # 需要利用网络进行学习的参数
static_args = {"uB_momentum":uB_momentum,"varB_momentum":varB_momentum}  # 需要在bn计算均值和方差时更新的参数
result = cal_bn(x,fn_grad,static_args,flag,eps)
print(result)